In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train ,y_test = train_test_split(df.drop('has_covid',axis=1),df['has_covid'],
                                                     test_size=0.2)

## 1.Aam Zindagi

In [14]:
# adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])
                                 
x_train_fever.shape

(80, 1)

In [18]:
# ordinal encoding => cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape


(80, 1)

In [21]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [22]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape


(80, 1)

In [24]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

## Mentos Zindagi

In [25]:
from sklearn.compose import ColumnTransformer

In [28]:
ct = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OneHotEncoder(sparse_output=False,drop='first'),['gender','city']),
    ('tnf3',OrdinalEncoder(categories=[['Mild','Strong']]),['cough'])
],remainder='passthrough')

### Read It :
   ColumnTransformer(transformers=[(transformer name , transformer class , columns to applay)],
                       remainder='what to do with other columns ? ('passthrough' / 'drop')

In [34]:
x_train_t=ct.fit_transform(x_train)

In [31]:
ct.transform(x_test).shape

(20, 7)

In [38]:
x_train_t

array([[100.95774648,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  23.        ],
       [101.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  38.        ],
       [ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   1.        ,  10.        ],
       [100.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   5.        ],
       [100.95774648,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  38.        ],
       [ 98.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  24.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   5.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [52]:
data = pd.DataFrame(x_train_t,columns=['fever','gender','city_D','city_K','city_M','cough','age'])
### converting array to dataframe

In [53]:
data.head()           ### transformed data set

,fever,gender,city_D,city_K,city_M,cough,age
0,100.957746,1.0,0.0,0.0,1.0,0.0,23.0
1,101.000000,0.0,1.0,0.0,0.0,0.0,64.0
2,101.000000,0.0,0.0,0.0,0.0,0.0,38.0
3,98.000000,0.0,0.0,1.0,0.0,1.0,10.0
4,100.000000,0.0,0.0,1.0,0.0,0.0,5.0


In [54]:
x_train.head()         ### old data set

,age,gender,fever,cough,city
25,23,Male,NaN,Mild,Mumbai
9,64,Female,101.0,Mild,Delhi
36,38,Female,101.0,Mild,Bangalore
99,10,Female,98.0,Strong,Kolkata
88,5,Female,100.0,Mild,Kolkata
